In [1]:
from IPython.core.debugger import set_trace

import numpy as np
import scipy as sp
import scipy.optimize as optim
from scipy.optimize import minimize
from scipy.misc import factorial
from scipy.special import polygamma, expit, logit, gammaln

import gdual as gd
import forward as fwd #for PGFs only

import matplotlib.pyplot as plt

In [ ]:
# theta_p = [7.0, 0.5]
# F_p = lambda s: fwd.negbin_pgf(s, theta_p)

theta_p = [6.0]
F_p = lambda s: fwd.poisson_pgf(s, theta_p)

# construct M_p, the MGF of p
M_p = lambda t: F_p(np.exp(t))

# use the MGF to compute the first k moments of p
k = 2
t0_gd = gd.GDual(0, q = k)
moments_p = M_p(t0_gd)
moments_p = moments_p.unwrap_coefs(moments_p.coefs, as_log = False)[1:] * factorial(np.arange(1, k+1))

print(moments_p)

mu_p  = moments_p[0]
var_p = moments_p[1] - (moments_p[0] ** 2)

print(mu_p)
print(var_p)

r_q = (mu_p ** 2) / (var_p - mu_p)
p_q = 1 - (mu_p/var_p)

print(r_q)
print(p_q)

In [4]:
def APGF_Poiss(mean_p):
    lambda_q = mean_p
    theta_q  = [lambda_q]
    
    return lambda s: fwd.poisson_pgf(s, theta_q)

def APGF_NB(mean_p, var_p):
    r_q = (mean_p ** 2) / (var_p - mean_p)
    p_q = 1 - (mean_p / var_p)
    theta_q = [r_q, p_q]
    
    return lambda s: fwd.negbin_pgf(s, theta_q)

def APGF(F_p):
    Z = F_p(1)
    
    # construct M_p, the MGF of p
    M_p = lambda t: F_p(np.exp(t))

    # use the MGF to compute the first k moments of p
    k = 2
    t0_gd = gd.GDual(0, q = k)
    moments_p = M_p(t0_gd)
    moments_p = moments_p.unwrap_coefs(moments_p.coefs, as_log = False)[1:] * factorial(np.arange(1, k+1))
    
    mean_p = moments_p[0]
    var_p  = moments_p[1] - (moments_p[0] ** 2)
    
    if mean_p == var_p:
        return APGF_Poiss(mean_p), Z
    else:
        return APGF_NB(mean_p, var_p), Z

In [ ]:
def APGF_forward(y,
                 immigration_pgf,
                 theta_immigration,
                 offspring_pgf,
                 theta_offspring,
                 rho,
                 GDualType=gd.LSGDual):
    K = len(y)
    
    A     = [None] * K # list of PGF function objects for each alpha message
    Gamma_hat = [None] * K # list of approximate Gamma PGFs
    
    # return Gamma_hat[k] if cached, otherwise compute it
    def Gamma(k):
        if Gamma_hat[k] is None:
            if k <= 0:
                Alpha_prev = lambda s: s ** 0#1.0
            else:
                Alpha_prev = lambda s: Alpha(s, k-1)
            
            F = lambda u:   offspring_pgf(u, theta_offspring[k-1])
            G = lambda u: immigration_pgf(u, theta_immigration[k])
            
            # construct Gamma_k, the true PGF of gamma_k
            Gamma_k = lambda u: Alpha_prev(F(u)) * G(u)
            
            # construct M_gamma_k, the MGF of gamma_k
            M_gamma_k = lambda t: Gamma_k(np.exp(t))
            
            # compute the mean and variance of gamma_k via the first two moments of gamma_k
            N_MOMENTS = 2
            zero_gd = GDualType(0, q = N_MOMENTS, as_log = False)
            moments = M_gamma_k(zero_gd)
            set_trace()
            moments = moments.unwrap_coefs(moments.coefs, as_log = False)[1:] * factorial(np.arange(1, N_MOMENTS + 1))

            mean_gamma_k = moments[0]
            var_gamma_k  = moments[1] - (moments[0] ** 2)
            
            # use the mean and variance to select and build an approximate PGF
            if mean_gamma_k == var_gamma_k:
                Gamma_hat[k] = APGF_Poiss(mean_gamma_k)
            else:
                Gamma_hat[k] = APGF_NB(mean_gamma_k, var_gamma_k)
        
        return Gamma_hat[k]
    
    def Alpha(s, k):
        Gamma_k = Gamma(k)
        
        const = s**y[k]
        const *= GDualType.const(y[k]*np.log(rho[k]) - gammaln(y[k] + 1), as_log=True)
        alpha = const * fwd.diff(Gamma_k, s*(1 - rho[k]), y[k], GDualType=GDualType )
        alpha.trunc_neg_coefs()
        A[k] = alpha
        return alpha
    
    A_final = lambda s: Alpha(s, K-1)
    
#     if d == 0:
#         alpha = A_final( 1.0 )
#     else:
#         alpha = A_final( GDualType(1.0, d) )
    alpha = A_final( GDualType(1.0, 1) )
    
    logZ = alpha.get(0, as_log=True)

    def marginals(k):
        a  = A_final( GDualType(0.0, k) )
        a /= GDualType.const(logZ, as_log=True)
        return a
        
    
    return logZ, alpha, marginals

y     = np.array([3,2,4])
lmbda = np.array([5,5,5]).reshape(-1,1)
delta = np.array([0.4, 0.4]).reshape(-1,1)
rho   = np.array([0.25, 0.25, 0.25])

logZ, alpha, marginals = APGF_forward(y,
                                     fwd.poisson_pgf,
                                     lmbda,
                                     fwd.bernoulli_pgf,
                                     delta,
                                     rho,
                                     GDualType=gd.LSGDual)


In [5]:
GDualType = gd.LSGDual

# def Gamma_k_builder(Alpha_k, F_k, G_k):
#     def Gamma_k(u_k): 
#         res = Alpha_k(F_k(u_k)) * G_k(u_k)
#         return res
#     return Gamma_k

# def Alpha_k_builder(Gamma_k, rho_k, y_k):
#     def Alpha_k(s_k):
#         const = s_k ** y_k
#         const *= GDualType.const(y_k * np.log(rho_k) - gammaln(y_k + 1), as_log = True)
#         res   = const * fwd.diff(Gamma_k, s*(1 - rho_k), y_k, GDualType = GDualType)
#         return res
#     return Alpha_k

def APGF_Forward(y,
                 immigration_pgf,
                 theta_immigration,
                 offspring_pgf,
                 theta_offspring,
                 rho):
    def Gamma_k(u_k, Alpha_PGF, k):
        #F(.) = offspring_pgf(u_k, theta_offspring[k-1])
        #G(.) = immigration_pgf(u_k, theta_immigration[k])
#         print(offspring_pgf(u_k, theta_offspring[k-1]))
        res = Alpha_PGF(offspring_pgf(u_k, theta_offspring[k-1])) * immigration_pgf(u_k, theta_immigration[k])
        return res
    
    def Alpha_k(s_k, Gamma_PGF, k):
        const = (s_k * rho[k])**y[k] / factorial(y[k])
#         print(Gamma_PGF)
        res = const * fwd.diff(Gamma_PGF, s_k*(1 - rho[k]), y[k], GDualType=GDualType)
        return res
    
    K = len(y)
    
    Gamma_PGFs = [None] * K
    Alpha_PGFs = [None] * (K+1)
    Z = [None] * K
    Alpha_PGFs[0] = lambda s_k: 1
    
    for i in range(K):
#         print(i)
        Gamma = lambda u_k,k=i: Gamma_k(u_k, Alpha_PGFs[k], k)
        res = APGF(Gamma)
        Gamma_PGFs[i] = res[0]
        Z[i] = res[1]
        
#         Gamma_PGFs[i] = APGF(Gamma)
        
        
#         print("%d.5"%i)
#         print(Gamma_PGFs)
#         print(Alpha_PGFs)
        
        Alpha_PGFs[i+1] = lambda s_k,k=i: Alpha_k(s_k, Gamma_PGFs[k], k)
    
    return Alpha_PGFs, Z

y     = np.array([3,2,4])
lmbda = np.array([5,5,5]).reshape(-1,1)
delta = np.array([0.4, 0.4]).reshape(-1,1)
rho   = np.array([0.25, 0.25, 0.25])

res = APGF_Forward(y,
                   fwd.poisson_pgf,
                   lmbda,
                   fwd.bernoulli_pgf,
                   delta,
                   rho)

print(res[0][-1](1))

# print(np.log(res[-1](1)))

[  4.99245092e-11]


/Users/kwinner/Work/latentcountmodels/python/forward.py:9: RuntimeWarning: divide by zero encountered in exp
  return np.exp(lmbda * (s - 1))


In [7]:
res2 = fwd.forward(y,
                   fwd.poisson_pgf,
                   lmbda,
                   fwd.bernoulli_pgf,
                   delta,
                   rho)
print(res2)

(-6.0515733053769445, array([ 0.00235416]), <function forward.<locals>.marginals at 0x10d1f8048>)


/Users/kwinner/Work/latentcountmodels/python/forward.py:9: RuntimeWarning: divide by zero encountered in exp
  return np.exp(lmbda * (s - 1))


In [ ]:
np.exp(5 * (gd.LSGDual.const(1) - 1))

In [6]:
print(np.sum([np.log(x) for x in res[1]]) + np.log(res[0][-1](1)))

[-35.66865295]


/Users/kwinner/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
